# CATBOOST TRAINING 

In [3]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import time
import pandas as pd 


In [4]:
X = pd.read_csv('../data/X.csv')
y = pd.read_csv('../data/y.csv')
X_test = pd.read_csv('../data/X_test.csv')

In [5]:
kf = KFold(n_splits=5, shuffle=True, random_state=52)

rmse_scores = []
test_predictions = np.zeros(len(X_test))
oof_predictions = np.zeros(len(X))  # Out-of-fold predictions

cat_features = ['Sex']  

model_params = {
    'iterations': 2000,
    'learning_rate': 0.02,
    'depth': 10,
    'l2_leaf_reg': 3,
    'random_seed': 52,
    'eval_metric': 'RMSE',
    'early_stopping_rounds': 100,
    'cat_features': cat_features,
    'verbose': 100
}


print("=== Training CatBoost ===")


for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
    print(f"\nFold {fold+1}")
    

    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[valid_idx], y.iloc[valid_idx]
    

    model = CatBoostRegressor(**model_params)
    

    start_time = time.time()
    

    model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        use_best_model=True
    )
    

    train_time = time.time() - start_time
    

    oof_predictions = model.predict(X_val)
    test_predictions = model.predict(X_test)
    
    

    fold_rmsle = np.sqrt(mean_squared_error(y_val, oof_predictions))
    rmse_scores.append(fold_rmsle)
    
    print(f"Fold {fold+1} RMSLE: {fold_rmsle:.4f}")
    print(f"Training time: {train_time:.1f} sec")


mean_rmsle = np.mean(rmse_scores)
std_rmsle = np.std(rmse_scores)
print(f"\nCatBoost Performance:")
print(f"Mean RMSLE: {mean_rmsle:.4f} ± {std_rmsle:.4f}")

=== Training CatBoost ===

Fold 1
0:	learn: 0.9448048	test: 0.9442074	best: 0.9442074 (0)	total: 121ms	remaining: 4m 1s
100:	learn: 0.1541874	test: 0.1543566	best: 0.1543566 (100)	total: 4.24s	remaining: 1m 19s
200:	learn: 0.0664084	test: 0.0669874	best: 0.0669874 (200)	total: 8.47s	remaining: 1m 15s
300:	learn: 0.0608279	test: 0.0614700	best: 0.0614700 (300)	total: 12.5s	remaining: 1m 10s
400:	learn: 0.0597972	test: 0.0606018	best: 0.0606018 (400)	total: 16.6s	remaining: 1m 6s
500:	learn: 0.0592322	test: 0.0602382	best: 0.0602382 (500)	total: 20.7s	remaining: 1m 1s
600:	learn: 0.0587874	test: 0.0600039	best: 0.0600039 (600)	total: 24.7s	remaining: 57.5s
700:	learn: 0.0584336	test: 0.0598613	best: 0.0598613 (700)	total: 28.7s	remaining: 53.2s
800:	learn: 0.0581117	test: 0.0597518	best: 0.0597518 (800)	total: 33.1s	remaining: 49.5s
900:	learn: 0.0578410	test: 0.0596629	best: 0.0596629 (900)	total: 37.1s	remaining: 45.3s
1000:	learn: 0.0575939	test: 0.0595991	best: 0.0595991 (1000)	total

## Submissions

In [ ]:
import os 


submission = pd.read_csv('../../dataset/sample_submission.csv')
predictions = np.expm1(test_predictions)

i = 1 
while os.path.exists(f"../predictions/submissions/cat_submission_{i}.csv"):
    i+=1

submission["Calories"] = predictions

csv_filename = f"../predictions/submissions/cat_submission_{i}.csv"
submission.to_csv(csv_filename, index=False)
print(f"Fichier CSV enregistré : {csv_filename}")

npy_filename = f"../predictions/oof/cat_oof_predictions_{i}.npy"
np.save(npy_filename, predictions)
print(f"Fichier NumPy enregistré : {npy_filename}")

Fichier CSV enregistré : ../predictions/submissions/submission_1.csv
Fichier NumPy enregistré : ../predictions/oof/cat_oof_predictions_1.npy
